# IITM RL FINAL PROJECT

## Problem - bsuite 

This notebook uses an open source reinforcement learning benchmark known as bsuite. https://github.com/deepmind/bsuite

bsuite is a collection of carefully-designed experiments that investigate core capabilities of a reinforcement learning agent.

Your task is to use any reinforcement learning techniques at your disposal to get high scores on the environments specified.

**Note**: Since the course is on Reinforcement Learning,  please limit yourself to using traditional Reinforcement Learning algorithms, 

**Do not use deep reinforcement learning.**

# How to use this notebook? 📝

- This is a shared template and any edits you make here will not be saved. **You
should make a copy in your own drive**. Click the "File" menu (top-left), then "Save a Copy in Drive". You will be working in your copy however you like.

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_RESULTS_DIR` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages

In [1]:
!pip install -q aicrowd-cli

     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 174kB 10.5MB/s 
     |████████████████████████████████| 204kB 14.3MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


# AIcrowd Runtime Configuration 🧷

Get login API key from https://www.aicrowd.com/participants/me


In [2]:
import os

AICROWD_RESULTS_DIR = os.getenv("OUTPUTS_DIR", "results")
os.environ["RESULTS_DIR"] = AICROWD_RESULTS_DIR
API_KEY = "6b42b562b9eb39fe9d07fb070f706ef5"

In [3]:
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Install packages 🗃

Please add all pacakage installations in this section

In [4]:
!pip install git+http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git
!pip install tabulate
!pip install tqdm

## Add any other installations you need here

  Cloning http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git to /tmp/pip-req-build-e37vk99e
  Running command git clone -q http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git /tmp/pip-req-build-e37vk99e
  Created wheel for bsuite: filename=bsuite-0.3.5-cp37-none-any.whl size=252042 sha256=ed5956815f7297219583483ef065e43d19e4d134857aff9d83a8525d5fb55b9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-c990illb/wheels/61/ea/06/77c82c07765fb8608e50e6c66bc566fa6d113c725bc6937e7b
Successfully built bsuite


# Import packages

In [5]:
import gym
import warnings

import numpy as np
import pandas as pd
import plotnine as gg
from tqdm.notebook import tqdm

import bsuite
from bsuite.aicrowd import environments
from bsuite.aicrowd.runner import Runner
from bsuite.aicrowd.analysis import Analyzer

import itertools
import math

pd.options.mode.chained_assignment = None
gg.theme_set(gg.theme_bw(base_size=16, base_family='serif'))
gg.theme_update(figure_size=(3, 1), panel_spacing_x=0.5, panel_spacing_y=0.5)
warnings.filterwarnings('ignore')

# **Agent Class**

You can modify the AGENT TEMPLATE below and implement the logic of your agent. Your agent must implement a few methods that will be called by the `Runner` class.
* `__init__` - put any initialization code here.
* `get_action` - takes in a `state` and returns an `action`.
* `learn` - takes in `(state, action, reward, next_state)`, implements the learning logic.
* `get_state` - takes in a raw `observation` directly from the env, discretizes it and returns a `state`.

In addition to these, you may implement other methods which can be called by the above methods.

Since there are multiple environments, you may need unique hyper parameters for each environment. Instantiate the agent while passing in the hyper parameters in a dictionary using the `agent_config` parameter so that each environment can use different hyper parameters for the agent while using a single `Agent` class for all of them.  You can use any names for the keys in the config dictionary.   

An example `RandomAgent` is given below.

In [6]:
# *** YOU CAN EDIT THIS CELL ***
# DO NOT rename the config dictionaries as the evaluator references them. However, you may use any names for the keys in them.
catch_config = {'env':environments.load_env(environments.CATCH),'name':'CATCH'}
catch_noise_config = {'env':environments.load_env(environments.CATCH_NOISE),'name':'CATCH_NOISE'}
cartpole_config = {'env':environments.load_env(environments.CARTPOLE),'name':'CARTPOLE'}
cartpole_noise_config = {'env':environments.load_env(environments.CARTPOLE_NOISE),'name':'CARTPOLE_NOISE'}
mountaincar_config = {'env':environments.load_env(environments.MOUNTAINCAR),'name':'MOUNTAINCAR'}
mountaincar_noise_config = {'env':environments.load_env(environments.MOUNTAINCAR_NOISE),'name':'MOUNTAINCAR_NOISE'}

Loaded bsuite_id: catch/0.
Loaded bsuite_id: catch_noise/1.
Loaded bsuite_id: cartpole/0.
Loaded bsuite_id: cartpole_noise/1.
Loaded bsuite_id: mountain_car/0.
Loaded bsuite_id: mountain_car_noise/1.


In [370]:
# *** YOU CAN EDIT THIS CELL ***
# AGENT TEMPLATE
class Agent:
    def __init__(self, agent_config=None):

        self.config = agent_config

        self.action_size = self.config['env'].action_space.n
        self.env = self.config['env']

        self.Q = {}
        self.states = []

        if ('CATCH' in self.config['name']):
          self.epsilon = 0.0001
          self.learning_rate = 0.1
          self.end_epsilon = 0.0001 
          self.end_learning_rate = 0.1
          self.exp_end_iter = 10000
          self.gamma = 0.99
          self.n_samples = 10000
          self.epsilon_decrement = ( self.epsilon - self.end_epsilon ) / self.exp_end_iter
          self.learning_rate_decrement = ( self.learning_rate - self.end_learning_rate ) / self.exp_end_iter
          self.catch_Q_Table_Init()
        elif ('CARTPOLE_NOISE' in self.config['name']):
          self.epsilon = 1
          self.learning_rate = 1
          self.end_epsilon = 0.1
          self.end_learning_rate = 0.1
          self.exp_end_iter = 1000
          self.gamma = 0.99995
          self.epsilon_decrement = ( self.epsilon - self.end_epsilon ) / self.exp_end_iter
          self.learning_rate_decrement = ( self.learning_rate - self.end_learning_rate ) / self.exp_end_iter
          self.cartpole_Q_Table_Init()
        elif ('CARTPOLE' in self.config['name']):
          self.epsilon = 1
          self.learning_rate = 0.1
          self.end_epsilon = 0.1
          self.end_learning_rate = 0.1
          self.exp_end_iter = 50
          self.gamma = 0.99
          self.epsilon_decrement = ( self.epsilon - self.end_epsilon ) / self.exp_end_iter
          self.learning_rate_decrement = ( self.learning_rate - self.end_learning_rate ) / self.exp_end_iter
          self.cartpole_Q_Table_Init()
        else:
          self.epsilon = 0.01
          self.learning_rate = 0.2
          self.end_epsilon = 0.01
          self.end_learning_rate = 0.2
          self.exp_end_iter = 1000
          self.gamma = 0.9999
          self.epsilon_decrement = ( self.epsilon - self.end_epsilon ) / self.exp_end_iter
          self.learning_rate_decrement = ( self.learning_rate - self.end_learning_rate ) / self.exp_end_iter
          self.cartpole_Q_Table_Init()


    def get_action(self, state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
        RETURNS     : 
            - action - 'action' to be taken
        '''

        if np.random.rand() <= self.epsilon:
          action =  np.random.choice(self.action_size)
        else:  
          qVals = [self.Q[(state,act)] for act in range(self.action_size)]
          action = qVals.index(max(qVals))

        return action
    
    def learn(self, state, action, reward, next_state, done):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
            - action - 'action' performed in 'state'
            - reward - 'reward' received due to action taken
            - next_state - discretized 'next_state'
        RETURNS     : 
            - NIL
        '''
        if (state == None) or (next_state == None):
          print(self.config['name'], "None Spotted")
          return

        next_q_values = [self.Q[(next_state,act)] for act in range(self.action_size)]
        target = reward + self.gamma * max(next_q_values)

        self.Q[(state,action)] += ( self.learning_rate * (target - self.Q[(state,action)]) )

        self.epsilon = max(self.end_epsilon, (self.epsilon - self.epsilon_decrement))
        self.learning_rate = max(self.end_learning_rate, (self.learning_rate - self.learning_rate_decrement))

    def get_state(self, observation):
        '''
        PARAMETERS  : 
            - observation - raw 'observation' from environment
        RETURNS     : 
            - state - discretized 'state' from raw 'observation'
        '''

        if ('CATCH' in self.config['name']):
          st = self.simplify_catch_state(observation)
          if st in self.states:
            state = st
          else:
            state = None
        else:
          state = self.transform_cartpole_state(self.simplify_cartpole_state(observation))

        return state

    def simplify_catch_state(self, observation):
      le = list(np.where((observation[:-1])==1))
      if len(le[0]) == 0:
        i = None
      else:
        i = le[0][0]
      if len(le[1]) == 0:
        j = None
      else:
        j = le[1][0]
      ne = list(np.where((observation[-1])==1))
      if len(ne[0]) == 0:
        k = 0
      else:
        k = ne[0][0]
      return (i,j,k)

    def catch_Q_Table_Init(self):

      done = False
      s = self.simplify_catch_state(self.env.reset())
      self.states.append(s)
      for _ in tqdm(range(self.n_samples)):
        a = np.random.choice(self.action_size)
        s2, _, done, _ = self.env.step(a)
        self.states.append(self.simplify_catch_state(s2))
        if done:
          s = self.simplify_catch_state(self.env.reset())
          self.states.append(s)

      for s in self.states:
        for a in range(self.action_size):
          #self.Q[(s,a)] = np.random.randn()
          self.Q[(s,a)] = 0
      print(len(self.Q))

    def simplify_cartpole_state(self, x):
      if ('MOUNTAINCAR' in self.config['name']):
        x = x[0][:-1]
      else:
        x = x[0][:-1]
        x = [math.atan2(x[2],x[3]),x[4]]
      return x

    def transform_cartpole_state(self, state):
      x = []
      for d in range(self.D):
        x.append(int(np.digitize(state[d], self.bins[d])))
      return tuple(x)

    def cartpole_Q_Table_Init(self):
      if ('CARTPOLE' in self.config['name']):
        if ('CARTPOLE_NOISE' in self.config['name']):
          n_bins = 17
        else:
          n_bins = 9
        max = np.array([[ 1., 5., 1., 1., 5., 1.]])
        min = np.array([[-1., -5., -1., -1., -5., 0.]])
        car_max = self.simplify_cartpole_state(max)
        #print(car_max)
        car_min = self.simplify_cartpole_state(min)
        #print(car_min)
        self.D = len(car_max)
        self.bins = []
        for _ in range(self.D):
          self.bins.append(np.linspace(car_min[_], car_max[_], num=n_bins))
        list_of_bins = []
        for d in range(self.D):
          list_of_bins.append(list(range(len(self.bins[d]) + 1)))
        for s in itertools.product(*list_of_bins):
          for a in range(self.action_size):
            self.Q[(s,a)] = 0
      else:
        n_bins = 14
        max = np.array([[ 0.6, 0.07, 1.]])
        min = np.array([[-1.2, -0.07, 0.]])
        car_max = self.simplify_cartpole_state(max)
        car_min = self.simplify_cartpole_state(min)
        self.D = len(car_max)
        self.bins = []
        for _ in range(self.D):
          self.bins.append(np.linspace(car_min[_], car_max[_], num=n_bins))
        list_of_bins = []
        for d in range(self.D):
          list_of_bins.append(list(range(len(self.bins[d]) + 1)))
        for s in itertools.product(*list_of_bins):
          for a in range(self.action_size):
            self.Q[(s,a)] = 0
      


In [371]:
RLAgent = Agent

In [372]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_config),
    env_id = environments.MOUNTAINCAR,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -426.0 	MEAN_REWARD:  -663.45 	EPISODE_LENGTH:  426
EPISODE:  200 	REWARD:  -240.0 	MEAN_REWARD:  -343.51 	EPISODE_LENGTH:  240
EPISODE:  300 	REWARD:  -224.0 	MEAN_REWARD:  -223.99 	EPISODE_LENGTH:  224
EPISODE:  400 	REWARD:  -155.0 	MEAN_REWARD:  -207.06 	EPISODE_LENGTH:  155
EPISODE:  500 	REWARD:  -172.0 	MEAN_REWARD:  -223.01 	EPISODE_LENGTH:  172
EPISODE:  600 	REWARD:  -164.0 	MEAN_REWARD:  -255.27 	EPISODE_LENGTH:  164
EPISODE:  700 	REWARD:  -181.0 	MEAN_REWARD:  -230.15 	EPISODE_LENGTH:  181
EPISODE:  800 	REWARD:  -306.0 	MEAN_REWARD:  -248.01 	EPISODE_LENGTH:  306
EPISODE:  900 	REWARD:  -225.0 	MEAN_REWARD:  -215.73 	EPISODE_LENGTH:  225
EPISODE:  1000 	REWARD:  -144.0 	MEAN_REWARD:  -252.32 	EPISODE_LENGTH:  144


In [373]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_noise_config),
    env_id = environments.MOUNTAINCAR_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -321.9843304051244 	MEAN_REWARD:  -721.86 	EPISODE_LENGTH:  323
EPISODE:  200 	REWARD:  -237.09025683824981 	MEAN_REWARD:  -236.49 	EPISODE_LENGTH:  242
EPISODE:  300 	REWARD:  -215.66412276747187 	MEAN_REWARD:  -215.04 	EPISODE_LENGTH:  219
EPISODE:  400 	REWARD:  -223.18503670199885 	MEAN_REWARD:  -237.5 	EPISODE_LENGTH:  219
EPISODE:  500 	REWARD:  -215.14699007778708 	MEAN_REWARD:  -240.67 	EPISODE_LENGTH:  216
EPISODE:  600 	REWARD:  -173.5895040353519 	MEAN_REWARD:  -261.91 	EPISODE_LENGTH:  172
EPISODE:  700 	REWARD:  -168.33398055604872 	MEAN_REWARD:  -203.27 	EPISODE_LENGTH:  167
EPISODE:  800 	REWARD:  -231.45341982506486 	MEAN_REWARD:  -204.55 	EPISODE_LENGTH:  228
EPISODE:  900 	REWARD:  -142.1058504092288 	MEAN_REWARD:  -170.41 	EPISODE_LENGTH:  146
EPISODE:  1000 	REWARD:  -151.07815263242185 	MEAN_REWARD:  -201.07 	EPISODE_LENGTH:  151


In [374]:
# *** PLEASE DONT EDIT THE CONTENTS OF THIS CELL ***
analyzer = Analyzer(os.environ.get('RESULTS_DIR'))
analyzer.print_scores()

╒════════════════════╤══════════╕
│ ENVIRONMENT        │    SCORE │
╞════════════════════╪══════════╡
│ catch              │ 0.993375 │
├────────────────────┼──────────┤
│ catch_noise        │ 0.98675  │
├────────────────────┼──────────┤
│ cartpole           │ 0.943028 │
├────────────────────┼──────────┤
│ cartpole_noise     │ 0.706642 │
├────────────────────┼──────────┤
│ mountain_car       │ 0.81375  │
├────────────────────┼──────────┤
│ mountain_car_noise │ 0.830592 │
╘════════════════════╧══════════╛


# **Playing with the Environment**

#### **Instantiating the environment** :
You can create an environment by calling the following function:  
`environments.load_env(ENV_ID)` - RETURNS: `env`  
where, ENV_ID can be ONE of the following:
* `environments.CATCH`
* `environments.CATCH_NOISE`
* `environments.CARTPOLE`
* `environments.CARTPOLE_NOISE`
* `environments.MOUNTAINCAR`
* `environments.MOUNTAINCAR_NOISE`

The `NOISE` environments add a scaled random noise to the `reward`.
<br/>

#### **Runnning the environment** :
There are certain methods required to run the environments. The interface is very similar to OpenAI Gym's interfaces. Fore more information, read the OpenAI documentation [here](https://gym.openai.com/docs/).

`env.reset()` - RETURNS: `observation`  
`env.step(action)`  - RETURNS: `(next_observation, reward, done, info[NOT USED])`

There are also a few useful properties within the environments:

* `env.action_space.n` - total number of possible actions. eg: if 'n' is 3, then the possible actions are `[0, 1, 2]`
* `env.observation_space.shape` -  the shape of the observation.
* `env.bsuite_num_episodes` -  the pre-specified number of episodes which will be run during evaluation (unique for each environment).

##### *ONLY IN CATCH / CATCH_NOISE*
* `env.observation_space.high` -  the upper limit for every index in the observation.
* `env.observation_space.low` -  the lower limit for every index of the observation.
<br/>


## **Environment Observation Space Limits:**

The limits for the observation space (minimum and maximum) for all the environments are given in the table below:

| Environments                        | Limits                                                                      |
|-------------------------------------|-----------------------------------------------------------------------------|
| CATCH <br/>  CATCH_NOISE            | MIN: use `env.observation_space.low` <br/> MAX: use `env.observation_space.high` |
| CARTPOLE <br/> CARTPOLE_NOISE       | MIN: `[-1. -5., -1., -1., -5., 0.]` <br/> MAX: `[ 1.,  5.,  1.,  1.,  5., 1.]` |
| MOUNTAINCAR <br/> MOUNTAINCAR_NOISE | MIN: `[-1.2, -0.07, 0.]` <br/> MAX: `[ 0.6,  0.07,  1.]`                                 |

[NOTE] Use this code cell to play around and get used to the environments. However, the `Runner` class below will be used to evaluate your agent.

In [10]:
runner = Runner(
    agent = RLAgent(agent_config=catch_config),
    env_id = environments.CATCH,
    log_interval = LOG_INTERVAL,
)
runner.play_episodes()

  0%|          | 0/10000 [00:00<?, ?it/s]

600
Loaded bsuite_id: catch/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  1.0 	MEAN_REWARD:  0.14 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  1.0 	MEAN_REWARD:  0.86 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.0 	MEAN_REWARD:  0.98 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1300 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1400 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1500 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISO

In [11]:
runner = Runner(
    agent = Agent(agent_config=catch_noise_config),
    env_id = environments.CATCH_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

  0%|          | 0/10000 [00:00<?, ?it/s]

600
Loaded bsuite_id: catch_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -0.4319180485080193 	MEAN_REWARD:  0.03 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  2.888194910660849 	MEAN_REWARD:  0.54 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  0.7058772924254098 	MEAN_REWARD:  0.55 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  0.5749511984405982 	MEAN_REWARD:  0.88 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.8165310035839968 	MEAN_REWARD:  1.07 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  0.39936420173707576 	MEAN_REWARD:  1.09 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.3884776851441103 	MEAN_REWARD:  0.91 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.8585504709523746 	MEAN_REWARD:  0.94 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.362278430457311 	MEAN_REWARD:  1.01 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.829857544425355 	MEAN_REWARD:  1.05 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  -0.37396752528475785 	MEAN_REWARD:  1.03 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  0.9410664532715053 	MEAN_REWARD:  1.04 	EPISODE_LENGTH:  9


In [12]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_config),
    env_id = environments.CARTPOLE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  1001.0 	MEAN_REWARD:  679.41 	EPISODE_LENGTH:  1001
EPISODE:  200 	REWARD:  432.0 	MEAN_REWARD:  971.09 	EPISODE_LENGTH:  433
EPISODE:  300 	REWARD:  1001.0 	MEAN_REWARD:  970.57 	EPISODE_LENGTH:  1001
EPISODE:  400 	REWARD:  1001.0 	MEAN_REWARD:  949.57 	EPISODE_LENGTH:  1001
EPISODE:  500 	REWARD:  1001.0 	MEAN_REWARD:  695.96 	EPISODE_LENGTH:  1001
EPISODE:  600 	REWARD:  455.0 	MEAN_REWARD:  842.6 	EPISODE_LENGTH:  456
EPISODE:  700 	REWARD:  1001.0 	MEAN_REWARD:  960.58 	EPISODE_LENGTH:  1001
EPISODE:  800 	REWARD:  1001.0 	MEAN_REWARD:  949.95 	EPISODE_LENGTH:  1001
EPISODE:  900 	REWARD:  1001.0 	MEAN_REWARD:  941.36 	EPISODE_LENGTH:  1001
EPISODE:  1000 	REWARD:  1001.0 	MEAN_REWARD:  899.46 	EPISODE_LENGTH:  1001


In [13]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_noise_config),
    env_id = environments.CARTPOLE_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  156.29957381303478 	MEAN_REWARD:  347.07 	EPISODE_LENGTH:  157
EPISODE:  200 	REWARD:  621.4747147158545 	MEAN_REWARD:  425.19 	EPISODE_LENGTH:  621
EPISODE:  300 	REWARD:  487.34227858242565 	MEAN_REWARD:  421.02 	EPISODE_LENGTH:  486
EPISODE:  400 	REWARD:  405.8087624463629 	MEAN_REWARD:  415.89 	EPISODE_LENGTH:  403
EPISODE:  500 	REWARD:  319.267671705993 	MEAN_REWARD:  410.7 	EPISODE_LENGTH:  318
EPISODE:  600 	REWARD:  355.0206387873918 	MEAN_REWARD:  411.03 	EPISODE_LENGTH:  357
EPISODE:  700 	REWARD:  381.3033360697864 	MEAN_REWARD:  415.27 	EPISODE_LENGTH:  383
EPISODE:  800 	REWARD:  566.554870609638 	MEAN_REWARD:  422.18 	EPISODE_LENGTH:  573
EPISODE:  900 	REWARD:  432.4681084414716 	MEAN_REWARD:  445.81 	EPISODE_LENGTH:  436
EPISODE:  1000 	REWARD:  597.8388857698641 	MEAN_REWARD:  419.57 	EPISODE_LENGTH:  600


## Point to the Agent Class you'll use for the final score

# **Evaluating the Agent on all the Environments**

* The following cells will take care of running your agent on each environment and aggregating the results in csv files. In each of the following cells, the `agent_config` parameter is already set to use the corresponding config dictionary for that environment. DO NOT EDIT THIS.
* Feel free to modify the `LOG_INTERVAL` parameter to change the interval between episodes for logging.  
* Please do not modify any other contents in each of the cells.  

In [9]:
LOG_INTERVAL = 100

# **Analysis & Result**

The following cells will show the score of the agent on each environment. The same scoring method will be used to evaluate your agent on a set of test environments.

In [ ]:
# If you want a object to get the scores
analyzer.get_scores()

{'cartpole': 0.05517450000000002,
 'cartpole_noise': 0.04140750000000003,
 'catch': 0.8470000000000001,
 'catch_noise': 0.823625,
 'mountain_car': 0.1,
 'mountain_car_noise': 0.13374400000000003}

# **Backend Evaluation**

THIS CODE WILL EVALUATE THE AGENT USING THE SPECIFIED CONFIGS FOR THE CORRESPONDING ENVIRONMENTS. DO NOT EDIT THE CONTENTS OF THIS CELL.

In [ ]:
## Do not edit this cell
if (os.environ.get('BACKEND_EVALUATOR') is not None):
    
    import backend_evaluator

    runs = {
        'catch': (
            backend_evaluator.CATCH, 
            catch_config),
        'catch_noise': (
            backend_evaluator.CATCH_NOISE, 
            catch_noise_config),
        'cartpole': (
            backend_evaluator.CARTPOLE, 
            cartpole_config),
        'cartpole_noise': (
            backend_evaluator.CARTPOLE_NOISE, 
            cartpole_noise_config),
        'mountaincar': (
            backend_evaluator.MOUNTAINCAR, 
            mountaincar_config),
        'mountaincar_noise': (
            backend_evaluator.MOUNTAINCAR_NOISE, 
            mountaincar_noise_config)
    }

    for run_name, run in runs.items():
        env_ids, config = run
        for env_id in env_ids:
            runner = Runner(env_id=env_id,
                            agent=RLAgent(agent_config=config),
                            verbose=False,
                            eval=True)
            runner.play_episodes()

# Submit to AIcrowd 🚀

**NOTE: PLEASE SAVE THE NOTEBOOK BEFORE SUBMITTING IT (Ctrl + S)**

In [ ]:
! aicrowd notebook submit --no-verify -c iitm-rl-final-project -a assets